Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [1]:

from random import random, seed
from itertools import product
import numpy as np
from matplotlib import pyplot as plt
from itertools import accumulate

from icecream import ic


## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [2]:
UNIVERSE_SIZE = 2000
NUM_SETS = 100
DENSITY = 0.1

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [3]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)

## Helper Functions

In [4]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution])) #phenotype: np.logical_or.reduce(SETS[solution])


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [ ]:
# A dumb solution of "all" sets
#solution = np.full(NUM_SETS, True)
#ic(valid(solution), cost(solution))
#None

In [ ]:
# A random solution with random 50% of the sets
#solution = rng.random(NUM_SETS) < .5
#ic(valid(solution), cost(solution))
#None

In [5]:
def fitness(solution:np.ndarray):
    return (valid(solution),-cost(solution))

In [6]:
# Apply the greedy algorithm to the first set of LIST_OF_SETS
class TabuSearch:
    def __init__(self, list_of_sets, max_iterations, tabu_tenure, max_no_improve):
        self.list_of_sets = list_of_sets
        self.max_iterations = max_iterations
        self.tabu_tenure = tabu_tenure
        self.max_no_improve = max_no_improve
        self.tabu_list = []
        self.best_solution = None
        self.best_cost = float('inf')
        self.evaluations=0

    def _get_coverage(self, selected_sets):
        # Calculate the coverage of the selected sets
        sets, _ = self.list_of_sets
        covered = np.any(sets[selected_sets], axis=0)
        return covered

    def _evaluate_solution(self, selected_sets):
        # Evaluate the solution based on the total cost of the selected sets
        self.evaluations += 1
        # Use the external cost function to compute the total cost
        total_cost = cost(selected_sets)
        # Get the coverage of elements by the selected sets
        covered_elements = self._get_coverage(selected_sets)
        # Calculate the number of uncovered elements
        num_uncovered = np.sum(~covered_elements)

        return total_cost, num_uncovered


    def _generate_initial_solution(self):
        # Greedy initialization: select sets that cover the most elements
        sets, _ = self.list_of_sets
        num_elements = sets.shape[1]
        uncovered_elements = np.ones(num_elements, dtype=bool)
        selected_sets = []

        while np.any(uncovered_elements):
            # Select the set that covers the most uncovered elements
            cover_count = np.sum(sets[:, uncovered_elements], axis=1)
            best_set = np.argmax(cover_count)
            selected_sets.append(best_set)
            uncovered_elements = uncovered_elements & ~sets[best_set]

        return selected_sets

    def _get_neighborhood(self, current_solution):
        # Generate neighborhood by adding or removing one set from the current solution
        sets, _ = self.list_of_sets
        num_sets = sets.shape[0]
        neighborhood = []

        # Try adding a new set
        for s in range(num_sets):
            if s not in current_solution:
                new_solution = current_solution + [s]
                neighborhood.append(new_solution)

        # Try removing a set
        for s in current_solution:
            new_solution = [i for i in current_solution if i != s]
            neighborhood.append(new_solution)

        return neighborhood

    def run(self):
        current_solution = self._generate_initial_solution()
        current_cost, num_uncovered = self._evaluate_solution(current_solution)
        best_solution = current_solution
        best_solution_cover=len(current_solution)-num_uncovered
        best_cost = current_cost
        no_improve_count = 0

        for iteration in range(self.max_iterations):
            # print(f"Iteration {iteration + 1}/{self.max_iterations}")
            neighborhood = self._get_neighborhood(current_solution)
            best_neigh_solution = None
            best_neigh_cost = float('inf')
            best_neigh_cover = 0

            # Evaluate all neighbors
            for neighbor in neighborhood:
                if neighbor not in self.tabu_list:
                    neigh_cost, neigh_uncovered = self._evaluate_solution(neighbor)
                    neigh_cover = len(neighbor) - neigh_uncovered  # Calculate coverage for neighbor

                    # Only consider valid solutions that cover all elements
                    if neigh_uncovered == 0 and neigh_cost < best_neigh_cost:
                        best_neigh_solution = neighbor
                        best_neigh_cost = neigh_cost
                        best_neigh_cover=neigh_cover#len(best_neigh_solution)-neigh_uncovered

            # If no valid neighbor found, stop
            if best_neigh_solution is None:
                break

            # Update current solution to best neighbor
            current_solution = best_neigh_solution
            current_cost = best_neigh_cost
            current_solution_cover=best_neigh_cover#len(current_solution)-neigh_uncovered


            # Update tabu list
            self.tabu_list.append(current_solution)
            if len(self.tabu_list) > self.tabu_tenure:
                self.tabu_list.pop(0)

            # Update best solution if necessary
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
                best_solution_cover=current_solution_cover

                no_improve_count = 0
            else:
                no_improve_count += 1

            # Stop if no improvement for too long
            if no_improve_count >= self.max_no_improve:
                break

        self.best_solution = best_solution
        self.best_cost = best_cost
        return best_solution, best_cost, self.evaluations, best_solution_cover #best_solution_cover num of set to cover all elements

In [7]:
# UNIVERSE_SIZE = 2000  # Number of elements in the universe
# NUM_SETS = 100        # Number of sets
# DENSITY = 0.1         # Approximate density of coverage (10% of elements covered by each set)

# Define your instances
instances = [
    {"Universe size": 100, "Num sets": 10, "Density": 0.2},
    {"Universe size": 1_000, "Num sets": 100, "Density": 0.2},
    {"Universe size": 10_000, "Num sets": 1_000, "Density": 0.2},
    {"Universe size": 100_000, "Num sets": 10_000, "Density": 0.1},
    {"Universe size": 100_000, "Num sets": 1_000, "Density": 0.2},
    {"Universe size": 100_000, "Num sets": 10_000, "Density": 0.3},
]

# Initialize TabuSearch parameters
max_iterations = 1000
tabu_tenure = 50
max_no_improve = 10

# Iterate over all instances
for i, instance in enumerate(instances, start=1):
    # Extract parameters
    universe_size = instance["Universe size"]
    num_sets = instance["Num sets"]
    density = instance["Density"]

    # Random number generator
    rng = np.random.Generator(np.random.PCG64())

    '''unedit line'''
    SETS = np.random.random((num_sets, universe_size)) < density
    for s in range(universe_size):
        if not np.any(SETS[:, s]):
            SETS[np.random.randint(num_sets), s] = True
    COSTS = np.power(SETS.sum(axis=1), 1.1)
    #print(f"1------------------")

    # Combine sets and costs in a tuple as input
    list_of_sets = (SETS, COSTS)

    #print(f"2------------------")

    # Initialize and run the TabuSearch algorithm
    tabu_search = TabuSearch(list_of_sets, max_iterations, tabu_tenure, max_no_improve) #self, list_of_sets, max_iterations, tabu_tenure, max_no_improve
    #print(f"3------------------")
    best_solution, best_cost, evaluations, best_solution_cover = tabu_search.run()
    #print(f"4------------------")

    # Output results for the current instance
    print(f"Results for Instance {i}:")
    print("Best Solution:", best_solution)
    print("Best Solution cover:", best_solution_cover)
    print("Best Cost:", best_cost)
    #print("Total Evaluations:", evaluations)
    print('-' * 40)  # Separator for clarity between instances


Results for Instance 1:
Best Solution: [1, 5, 6, 3, 9, 4, 2, 7, 0, 8]
Best Solution cover: 10
Best Cost: 282.3050319470386
----------------------------------------
Results for Instance 2:
Best Solution: [89, 97, 99, 10, 50, 11, 64, 75, 88, 27, 66, 32, 77, 6, 98, 0, 35]
Best Solution cover: 17
Best Cost: 6196.1924046211825
----------------------------------------
Results for Instance 3:
Best Solution: [635, 323, 542, 202, 747, 448, 679, 425, 45, 692, 600, 599, 210, 223, 737, 147, 410, 57, 656, 292, 401, 930, 220]
Best Solution cover: 23
Best Cost: 99857.84962581997
----------------------------------------
Results for Instance 4:
Best Solution: [9967, 6820, 2258, 4554, 2614, 9531, 5486, 9356, 4232, 7764, 3255, 6246, 7435, 7206, 1824, 4575, 1116, 2034, 788, 2586, 8309, 4756, 2550, 108, 2952, 8510, 1811, 384, 1480, 3865, 4625, 8091, 7050, 3894, 5768, 474, 8712, 9856, 185, 3684, 3762, 4284, 2869, 7223, 5949, 1386, 5110, 2035, 8727, 7066, 7620, 1638, 3578, 4474, 3447, 8601, 2921, 1492, 1683,